In [1]:
'''
数据清洗主程序
'''

import pandas as pd
import post_clean as pc

In [2]:
df =  pd.read_csv("final.csv",index_col=0)
print(df.shape)
df.drop_duplicates(inplace=True) # 删除重复行 16万行剩下3万7
df.reset_index(drop=True,inplace=True) # 重置列号
print(df.shape)
print(df.info())
df.head(5)

(161450, 10)
(37509, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37509 entries, 0 to 37508
Data columns (total 10 columns):
职位      37509 non-null object
公司      37509 non-null object
工作地点    37509 non-null object
薪水      36420 non-null object
发布日期    37509 non-null object
职位信息    37444 non-null object
福利      31359 non-null object
学历      30454 non-null object
经验年数    37509 non-null object
招聘人数    37509 non-null object
dtypes: object(10)
memory usage: 2.9+ MB
None


,职位,公司,工作地点,薪水,发布日期,职位信息,福利,学历,经验年数,招聘人数
0,ICBU-国际贸易事业部-资深服务运营专员-杭州,阿里巴巴集团,杭州,NaN,10-18,\r\r\n\t\t\t\t\t\t1、通过科学有效的方法提高渠道商的专业/服务能力，判断进...,NaN,本科,5-7年经验,招2人
1,ERP实施顾问(实习岗）,金蝶软件合肥分公司,合肥,4-6千/月,10-18,\r\n1、帮助客户成功应用ERP软件，根据实施方法论，结合客户管理信息化需求，主导ERP项...,"绩效奖金,定期体检,员工旅游,年终奖金,弹性工作,专业培训",本科,无工作经验,招5人
2,数据仓库工程师,土巴兔装修网,深圳-南山区,1.5-3万/月,10-18,\r\n岗位职责：1.负责企业级数据仓库的模型设计；2.负责数据仓库及BI项目基础架构规划、...,"五险一金,员工旅游,交通补贴,绩效奖金,股票期权,弹性工作,定期体检,专业培训",NaN,无工作经验,招1人
3,测试工程师/测试开发工程师,证通股份有限公司,上海-浦东新区,1.5-3万/月,10-18,\r\n职责范围：* 负责项目/产品测试计划、测试方案和测试用例的制定、设计、编写和评审；*...,"做五休二,补充公积金,补充医疗保险,节日福利,免费班车,餐饮补贴",本科,无工作经验,招2人
4,Altium技术支持工程师,北京亿道电子技术有限公司,北京-朝阳区,0.6-1.5万/月,10-18,\r\n一、岗位职责：工作职责：1、负责Altium PCB产品线的解决方案；2、负责为客户...,"做五休二,周末双休,弹性工作,五险一金,绩效奖金,节日福利,专业培训,员工旅游",本科,1年经验,招1人


In [3]:
# 1.职位信息和发布日期在后续的分析中用不到，删除
del df['职位信息']
del df['发布日期']

In [4]:
# 2.调用post_clean.clean()对职位名称进行清理
print('职位名称个数为'+str(df.职位.value_counts().size))
pc.clean(df)
print('职位名称个数为'+str(df.职位.value_counts().size))

职位名称个数为20955
职位名称个数为4149


In [5]:
# 3.删掉需求量小于3的职位的招聘信息，避免个别数据对分析结果的影响
print(df.shape)
a=[]
se = df.职位.value_counts()
for i in se.index:
    if se[i]<3:
        a.append(i)
print(len(a))
df.drop(df[df.职位.isin(a)].index,inplace=True)
print(df.shape)

(37509, 8)
3589
(33564, 8)


In [6]:
"""
4.将薪水格式化，将薪水转化为最高工资和最低工资，并统一单位为万/月
首先通过查看dataframe信息发现薪水一列中存在空值，则先将空值除去
然后通过观察数据源发现，薪水的单位有四种，分别为 千/月，万/月，万/年，元/天
通过找到其索引，将其进行分割，找到最低和最高工资的值，将其分离到新的列中，分别为最低工资和最高工资
"""
#去空
index0 = df[df.薪水.isnull()].index
df.drop(axis=0,index=index0,inplace=True)

#将薪水分类，万/年，万/月，千/月，元/天
index1 = df[df['薪水'].str.contains("千/月")].index    #87
index2 = df[df['薪水'].str.contains("万/月")].index      #439
index3 = df[df['薪水'].str.contains("万/年")].index     #13
index4 = df[df['薪水'].str.contains("元/天")].index      #3

df['最低工资'] = 0.0
df['最高工资'] = 0.0

for i in index1:
    money = []
    money.extend(df.loc[i]['薪水'].split("-")) 
    df.at[i,'最低工资'] = round(float(money[0])/10,2)
    df.at[i,'最高工资'] = round(float(money[1][:-3])/10,2)
for i in index2:
    money = []
    money.extend(df.loc[i]['薪水'].split("-"))
    df.at[i,'最低工资'] = round(float(money[0]),2)
    df.at[i,'最高工资'] = round(float(money[1][:-3]),2)
for i in index3:
    money = []
    money.extend(df.loc[i]['薪水'].split("-"))
    df.at[i,'最低工资'] = round(float(money[0])/12,2)
    df.at[i,'最高工资'] = round(float(money[1][:-3])/12,2)
for i in index4:
    money = []
    money.extend(df.loc[i]['薪水'].split("-"))
    df.at[i,'最低工资'] = round(float(money[0][:-3])*30/10000,2)
    df.at[i,'最高工资'] = round(float(money[0][:-3])*30/10000,2)
del df['薪水'] # 处理结束，删除该列

In [7]:
"""
5.对工作地点进行格式化处理
首先通过查看dataframe信息发现地点一列中不存在空值
其次通过观察数据源，发现地点这一属性的规律，"-"左边是市级地点，右边是区级地点
我们只需要市级地点，故只需要将"-"及其后的去掉即可
"""
df['工作地点']=df['工作地点'].apply(lambda x: x.split(sep='-')[0])

In [8]:
"""
6.将招聘人数这一列格式化，并转化为整型
首先通过查看dataframe信息发现该列中不存在空值
其次通过观察数据源，发现该列的规律，有“招×人”和“招若干人”两种说法
我们通过字符串的切片，将“招”“人”去掉
之后计算包含数字的招聘人数的平均值
用招聘人数的平均值来填充“招若干人”的数据
"""
#将招聘人数这一列转化为整型
df['招聘人数'] = df['招聘人数'].apply(lambda x:x[1:-1])
num = int(df[df['招聘人数']!= '若干']['招聘人数'].apply(lambda x:int(x)).mean())
df['招聘人数'] = df['招聘人数'].apply(lambda x:num if x=='若干' else int(x))

In [9]:
"""
7.处理经验年数这一列，对其进行格式化处理
首先发现无空值
通过观察数据源，发现该列的格式有以下几种：
无工作经验
×年经验
3-4年经验
5-7年经验
8-9年经验
10年以上经验
对其尽行整型化处理
"""
#处理经验年数
def to_int(x):
    if x == "无工作经验":
        return float(0)
    elif x == '3-4年经验':
        return float(3.5)
    elif x == '5-7年经验':
        return float(6)
    elif x == '8-9年经验':
        return float(8.5)
    elif x =='10年以上经验':
        return float(10)
    else:
        return float(x[:-3])
df['经验年数'] = df['经验年数'].apply(to_int)

In [10]:
"""
8.对学历这一列进行处理
发现学历这一列中有空值，要对其进行填充
用“无学历要求”来填充空值
"""
df.学历 = df.学历.fillna("无学历要求")

In [11]:
"""
9.对福利这一列进行数字化处理
我们想要将福利这一列的文字信息可以量化表示
于是将不同项目的福利信息进行了量化，赋予其不同的权值，值越高表示福利待遇越好
首先将为空的数据用“无”填充，方便下一步的处理
将大部分出现的福利类型（寻找不同句子中相似的成分）与其相应的权值构造为一个字典
通过福利这一项中的字符串与不同的福利类型相匹配从而得到每一列的福利值
新增一列为福利值，为数值类型，方便接下来的分析
"""
df.福利 = df.福利.fillna("无")
global welfare 
global welfare_list
welfare = set()      #用于存储所有福利的类型（无重复）
welfare_list = []    #用于存储所有出现的福利（有重复）
def add_welfare(x):
    global welfare
    global welfare_list
    if x!='无':
        welfare.update(x.split(","))
        welfare_list.extend(x.split(","))
df.福利.apply(add_welfare)
num = []
for item in welfare:
    count = welfare_list.count(item)
    num.append(count)

num = []
for item in welfare:
    count = welfare_list.count(item)
    num.append(count)

# wel = pd.Series(num,index=welfare)
# wel.sort_values(ascending=False).head(80)

#提取出福利待遇相似的部分，为其分配权值
global welfare_dic
welfare_dic = {
    "三险":65,
    "四险":70,
    "五险一金":80,
    "五险二金":85,
    "五险三金":90,
    "六险一金":85,
    "六险二金":90,
    "七险一金":90,
    "八险一金":95,
    "交通补贴":20,
    "通讯补贴":20,
    "餐饮补贴":15,
    "加班补贴":30,
    "加班补助":30,
    "高温补贴":10,
    "住房补贴":40,
    "租房补贴":20,
    "免费三餐":30,
    "免费体检":10,
    "免费专业体检":15,
    "免费班车":25,
    "班车接送":25,
    "免费零食":10,
    "免费中餐":15,
    "免费午餐":15,
    "免费下午茶":10,
    "绩效奖金":40,
    "年终奖金":60,
    "全勤奖":30,
    "孝敬父母奖":10,
    "包吃":30,
    "包三餐":30,
    "包住":50,
    "包食宿":80,
    "提供住宿":50,
    "生日":10,
    "双休":50,
    "做五休二":60,
    "做六休一":10,
    "带薪年假":30,
    "带薪休假":35,
    "带薪培训":40,
    "年底双薪":30,
    "年底十三薪":20,
    "假日":20,
    "节日":20,
    "节假日":20,
    "产假":50,
    "保险":20,
    "股票期权":50,
    "补充公积金":20,
    "弹性":10,
    "健身":10,
    "旅游":30,
    "培训":30,
    "出国":30,
    "团建":10,
    "扁平":10,
    "下午茶":10,
    "团队氛围好":20,
    "过亿用户":30,
    "晋升空间":35,
    "职业大牛":35,
    "聚餐":10,
    "社保":20
}

# df[df.福利.str.contains("节日")].福利

def compute_welfare(x):
    value = 0
    if x=="无":
        value = 0
    else:
        length = len(x.split(","))
        value += length*1
    global welfare_dic
    for item in welfare_dic.keys():
        if x.find(item)!=-1:
            value += welfare_dic[item]
    #print(value)
    return value
df["福利值"] = 0 
df["福利值"] = df.福利.apply(compute_welfare)

In [12]:
df.head(5)

,职位,公司,工作地点,福利,学历,经验年数,招聘人数,最低工资,最高工资,福利值
1,ERP实施,金蝶软件合肥分公司,合肥,"绩效奖金,定期体检,员工旅游,年终奖金,弹性工作,专业培训",本科,0.0,5,0.4,0.6,176
2,数据仓库工程师,土巴兔装修网,深圳,"五险一金,员工旅游,交通补贴,绩效奖金,股票期权,弹性工作,定期体检,专业培训",无学历要求,0.0,1,1.5,3.0,268
3,软件测试工程师,证通股份有限公司,上海,"做五休二,补充公积金,补充医疗保险,节日福利,免费班车,餐饮补贴",本科,0.0,2,1.5,3.0,166
6,C#.NET开发工程师,武汉微诚科技股份有限公司西安研发中心,西安,"保险,交通补贴,餐饮补贴,绩效奖金,年终奖金,专业培训",本科,6.0,5,0.6,1.5,191
7,C++研发工程师,北京凯罗斯软件科技有限公司,北京,无,本科,0.0,2,1.0,1.5,0


In [13]:
# 将处理后的数据集输出为csv文件，方便接下来的使用
df.to_csv("Temp.csv",index=False)
df.shape

(32805, 10)